# Рекомендация тарифов

## Описание проекта

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».
В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением accuracy. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте accuracy на тестовой выборке самостоятельно.

**Инструкция по выполнению проекта**

1. Откройте файл с данными и изучите его. Путь к файлу: datasets/users_behavior.csv
1. Разделите исходные данные на обучающую, валидационную и тестовую выборки.
1. Исследуйте качество разных моделей, меняя гиперпараметры. Кратко напишите выводы исследования.
1. Проверьте качество модели на тестовой выборке.
1. Дополнительное задание: проверьте модели на вменяемость. Ничего страшного, если не получится: эти данные сложнее тех, с которыми вы работали раньше. В следующем курсе подробнее об этом расскажем.

**Описание данных**

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц.

Известно:
1. сalls — количество звонков,
1. minutes — суммарная длительность звонков в минутах,
1. messages — количество sms-сообщений,
1. mb_used — израсходованный интернет-трафик в Мб,
1. is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## 1. Откройте и изучите файл

In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score
import timeit

SEED = 21

In [2]:
data = pd.read_csv('/datasets/users_behavior.csv')
data.sample(2)

,calls,minutes,messages,mb_used,is_ultra
2348,141.0,1102.88,50.0,16951.74,0
2725,44.0,342.11,25.0,18110.91,0


In [3]:
data['is_ultra'].value_counts()

0    2229
1     985
Name: is_ultra, dtype: int64

Видим что выборка не сбалансирована, перешедших на тариф "Ультра" в ~2.3 раза меньше

Разделим данные на признаки и целевое значение

In [4]:
X = data.drop('is_ultra', axis=1)
y = data['is_ultra']

In [5]:
X.shape, y.shape

((3214, 4), (3214,))

## 2. Разбейте данные на выборки

Разобьем данные на train и test, на стартифицированные выборки, сохраним дисбаланс классов.

Валидационную выборку выделять не будем, качество проверим на кросс валидации. Подготовим фолды для валидации

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED, stratify=y)

Проверим что дисбаланс сохранился в трейне и в тесте

In [7]:
np.isclose((y_train==0).sum()/y_train.shape[0], (y_test==0).sum()/y_test.shape[0], 0.001)

True

In [8]:
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=SEED)

## 3. Исследуйте модели

### Дерево решений
Начнем с дерева решений, подберем гиперпараметр `max_depth` на кросс валидации.

In [10]:
models = []
params = []

dt = DecisionTreeClassifier(random_state=SEED,)
params_dt = {'max_depth': np.linspace(1, 10, 10).astype(int)}

models.append(dt)
params.append(params_dt)

Обучим дерево на всей train выборке с лучшими параметрами и сделаем предсказания для теста

### Случайный лес
Попробуем случайный лес, настраивать будем количесво деревьев `n_estimators` и количество используемых признаков `max_features `

In [11]:
rf = RandomForestClassifier(random_state=SEED)
params_rf = {'n_estimators': np.linspace(10, 100, 10).astype(int),
             'max_features': list(range(1, X.shape[1]+1))
            }

models.append(rf)
params.append(params_rf)

<font color=green>Все корректно с подбором параметров, но помни про неодинаковый cv:)</font>

### Логистическая регрессия
Подготовим модель логистической регрессии, подбирать будем коэффициент регуляризации `C`

In [12]:
logit = LogisticRegression(random_state=SEED, solver='liblinear')
params_logit = {'C': np.logspace(-2, 1, 10)}

models.append(logit)
params.append(params_logit)

### Сравним модели
Запустим ранее написанную функцию на всех моделях, сведем все данные в одну таблицу

In [13]:
cols = ['mean_fit_time', 'mean_test_score', 'std_test_score', 'params']
d = []

for m, p in zip(models, params):
    d.append(best_model(m, p, X_train, y_train, cv)[cols])

res = pd.DataFrame(data=d, index=['Tree', 'Forest', 'Logit'], columns=cols)
res

,mean_fit_time,mean_test_score,std_test_score,params
Tree,0.007300,0.793466,0.015161,{'max_depth': 8}
Forest,0.128672,0.798133,0.014895,"{'max_features': 1, 'n_estimators': 40}"
Logit,0.010278,0.716842,0.011435,{'C': 0.21544346900318834}


Из таблице выше можно увидеть, что лучшие результаты у случаного леса, но при этом он самый долгий.

Остановимся на нем, так как в данном случае считаем, что его время работы не критично.

От дерева откажемся из-за его склонности к переобучению, хотя по std этого не скажешь.

## 4. Проверьте модель на тестовой выборке

Проверим выбранную модель на тестовой выборке

In [14]:
rf_params = res.loc['Forest', 'params']
rf = RandomForestClassifier(**rf_params, random_state=SEED)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test, y_pred)

0.8164852255054432

Получили долю правильных ответов даже выше чем на cv.

## 5. (бонус) Проверьте модели на адекватность

Будем считать, что если модель предсказывает лучше чем простое предположение, что количество перешедишх на тариф "смарт" и "ультра", будет с такими же пропорциями что и в исходной выборке, то она адекватна

Посчитаем количество перешедших на тариф "смарт" к общему количеству объектов в выборке.

In [15]:
(data['is_ultra']==0).sum() / data.shape[0]

0.693528313627878

Мы видим что доля правильных ответов моделей выше чем у нашего наивного предположения, считаем модели адекватными.